In [83]:
# general
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

%matplotlib inline

from pylab import rcParams
rcParams['figure.figsize'] = 8, 5

# first used in exercise one
import linearsvm as svm
from sklearn import preprocessing # for scale
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [19]:
import importlib
importlib.reload(svm)

<module 'linearsvm' from '/Users/andrewenfield/work/github/Data558/Week08/linearsvm.py'>

Note: Per the request in the "Collaboration policy" note, I've discussed at least part of this assignment with many of the MS employees in the class, including Abhishek, Geoff, Suman, and Charles. (Different weeks/different assignments have different people, depending upon who attends our study groups, but I'll probably just include this blurb w/ each homework since it's generally correct.) I've also gotten input from the discussion board.

# Exercise one

_Compute the gradient ∇F(β) of F._

![Gradient](gradient.png)

_Consider the Spam dataset from The Elements of Statistical Learning. Standardize the data, if you have not done so already._

In [40]:
spam = pd.read_table('data/spam.data', sep=' ', header=None)
spam.shape

(4601, 58)

In [41]:
spam[:2]

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.778,0.00,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.0,0.94,...,0.0,0.132,0.0,0.372,0.18,0.048,5.114,101,1028,1


In [42]:
spam[57].value_counts(dropna=False)

0    2788
1    1813
Name: 57, dtype: int64

In [43]:
# convert 0/1 to -1/1
spam[57] = spam[57].apply(lambda v: -1 if v == 0 else 1)

In [44]:
spam[57].value_counts(dropna=False)

-1    2788
 1    1813
Name: 57, dtype: int64

In [45]:
X = spam.values[:,0:57]
y = spam.values[:,57]
X.shape, y.shape

((4601, 57), (4601,))

In [47]:
X_scaled = preprocessing.scale(X)
X_scaled.shape

(4601, 57)

In [48]:
X_scaled[:1]

array([[-0.34243371,  0.3308849 ,  0.71285877, -0.04689958,  0.01156471,
        -0.35026618, -0.29179389, -0.26256156, -0.32330236, -0.37136439,
        -0.29685953,  0.11408733, -0.31205521, -0.17492717, -0.19011441,
         0.08617144, -0.32113541,  2.08120664,  0.15090537, -0.16789311,
         0.1251284 , -0.11817151, -0.2902092 , -0.21299439, -0.32881467,
        -0.29923993, -0.22789481, -0.23183016, -0.16673145, -0.22523952,
        -0.16053931, -0.14321202, -0.17492026, -0.14521515, -0.19806739,
        -0.24213022, -0.32345561, -0.05983624, -0.18091134, -0.18530385,
        -0.12090468, -0.17259996, -0.20599311, -0.12734332, -0.29777621,
        -0.19738748, -0.0713879 , -0.11154623, -0.15845336, -0.51430655,
        -0.15519768,  0.62400658, -0.30835494, -0.1030484 , -0.04524728,
         0.04529792, -0.00872413]])

_Write a function mylinearsvm that implements the fast gradient algorithm to train the linear support vector machine with the squared hinge loss. The function takes as input the initial step-size value for the backtracking rule and a maximum number of iterations._

I implemented the mylinearsvm function, and all of the supporting functions including the gradient and objective functions, in the file linearsvm.py, which I imported into this notebook with the alias svm.

_Train your linear support vector machine with the squared hinge loss on the the Spam dataset for the λ = 1. Report your misclassiﬁcation error for this value of λ._ 

In [49]:
results = svm.fastgradalgo(
    X_scaled, y, t_init=0.01, 
    grad_func = svm.compute_linearsvm_gradient, 
    obj_func = svm.compute_linearsvm_objective, 
    lam=1)
results[-5:]

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
96,0.004773,-0.013833,0.034621,0.021870,0.070751,0.052163,0.114484,0.057632,0.047464,0.025636,...,-0.022830,-0.025820,-0.012043,-0.011067,0.068568,0.101598,0.023084,0.025797,0.056279,0.063825
97,0.004795,-0.013829,0.034627,0.021865,0.070724,0.052143,0.114499,0.057609,0.047451,0.025639,...,-0.022822,-0.025824,-0.012051,-0.011070,0.068546,0.101618,0.023089,0.025803,0.056276,0.063813
98,0.004815,-0.013825,0.034634,0.021860,0.070697,0.052127,0.114510,0.057587,0.047441,0.025644,...,-0.022816,-0.025823,-0.012061,-0.011072,0.068525,0.101640,0.023095,0.025809,0.056275,0.063798
99,0.004834,-0.013819,0.034642,0.021853,0.070672,0.052116,0.114516,0.057567,0.047434,0.025650,...,-0.022811,-0.025817,-0.012072,-0.011071,0.068505,0.101663,0.023100,0.025815,0.056274,0.063782
100,0.004849,-0.013813,0.034652,0.021845,0.070649,0.052109,0.114517,0.057549,0.047430,0.025657,...,-0.022808,-0.025807,-0.012084,-0.011070,0.068488,0.101688,0.023105,0.025819,0.056273,0.063765


In [50]:
svm.get_final_coefs(results)

array([[ 0.00484856, -0.01381278,  0.03465152,  0.02184484,  0.07064941,
         0.05210909,  0.11451729,  0.05754886,  0.04742959,  0.02565736,
         0.0395877 , -0.02027215,  0.01739698,  0.00703854,  0.03922624,
         0.08387392,  0.06459502,  0.05084026,  0.04901419,  0.04912297,
         0.08789593,  0.04203993,  0.09554004,  0.06106319, -0.05455013,
        -0.03976008, -0.0484804 , -0.01311055, -0.01641889, -0.02634323,
        -0.01025193, -0.00084591, -0.03294699, -0.00048894, -0.02046245,
        -0.00402314, -0.03110056, -0.01247296, -0.02326556,  0.0067796 ,
        -0.01671461, -0.03714934, -0.02274022, -0.02592255, -0.04297767,
        -0.04192052, -0.01662661, -0.02280837, -0.0258065 , -0.01208367,
        -0.01106978,  0.0684883 ,  0.1016875 ,  0.02310485,  0.02581931,
         0.05627304,  0.06376549]])

In [89]:
def get_accuracy(beta_coefs, X, y, threshold=0):
    y_pred = X.dot(beta_coefs.T).ravel() # ravel to convert to vector
    # convert to -1 or +1 depending on threshold
    y_thresholded = np.where(y_pred > threshold, 1, -1)
    return accuracy_score(y, y_thresholded)

In [96]:
accuracy = get_accuracy(svm.get_final_coefs(results), X_scaled, y)
print("Accuracy: {0:.1%}".format(accuracy))
print("Misclassification error: {0:.1%}".format(1 - accuracy))

Accuracy: 91.1%
Misclassification error: 8.9%


**TODO** ? try with a basic sklearn impl and see if coefs and accuracy are close?

_Run cross-validation to ﬁnd the optimal value of λ. Report your misclassiﬁcation error for that value of λ._

In [114]:
def train_and_test_single_fold(X_full, y_full, lam, train_index, test_index):
    """
    Train using the data identified by the indices in train_index, and then test
    (and return accuracy) using the data identified by the indices in test_index.
    """
    beta_vals = svm.fastgradalgo(
        X_full[train_index], y_full[train_index], t_init=0.01, 
        grad_func = svm.compute_linearsvm_gradient, 
        obj_func = svm.compute_linearsvm_objective, 
        lam=lam)

#     beta_vals = mt.randcoorddescent(X_full[train_index], y_full[train_index], 
#                                     lam, alpha, max_iter=500, beta0=beta0)

    final_coefs = svm.get_final_coefs(beta_vals)
    
    return get_accuracy(final_coefs, X_full[test_index], y_full[test_index])
    
#     return mean_squared_error(y_full[test_index], 
#                               X_full[test_index].dot(final_coefs))

In [115]:
def train_and_test_for_all_folds(X_full, y_full, train_indices, 
                                 test_indices, lam):
    """
    Train and test for all folds - for now, 10 folds, hard-coded. Return 
    the mean of the set of accuracy scores from all folds."""
    accuracy_scores = [train_and_test_single_fold(X_full, y_full, lam,
                                       train_indices[i], 
                                       test_indices[i]) for i in range(10)]
    return(np.mean(accuracy_scores))

In [116]:
# get arrays with 10 sets of test and train indices - one for each fold
kf = KFold(10, shuffle=True, random_state=42)

train_indices_list = []
test_indices_list = []
for train_index, test_index in kf.split(X_scaled):
    train_indices_list.append(train_index)
    test_indices_list.append(test_index)
    
train_indices = np.array(train_indices_list)
test_indices = np.array(test_indices_list)

In [126]:
lambdas = [10 ** exponent for exponent in range(-15,3)]
lambdas

[1e-15,
 1e-14,
 1e-13,
 1e-12,
 1e-11,
 1e-10,
 1e-09,
 1e-08,
 1e-07,
 1e-06,
 1e-05,
 0.0001,
 0.001,
 0.01,
 0.1,
 1,
 10,
 100]

In [127]:
# lambdas = np.logspace(-5, 0.1, num=10)
# lambdas

In [128]:
# and finally, do 10-fold cross validation for each value of lambda, and
# show the mean of each set's MSEs
accuracy_values_by_lambda = [train_and_test_for_all_folds(X_scaled, y, 
                                train_indices, test_indices, 
                                lam) for lam in lambdas]
list(zip(lambdas, accuracy_values_by_lambda))

[(1e-15, 0.9169777421484484),
 (1e-14, 0.9169777421484484),
 (1e-13, 0.9169777421484484),
 (1e-12, 0.9169777421484484),
 (1e-11, 0.9169777421484484),
 (1e-10, 0.9169777421484484),
 (1e-09, 0.9169777421484484),
 (1e-08, 0.9169777421484484),
 (1e-07, 0.9169777421484484),
 (1e-06, 0.9169777421484484),
 (1e-05, 0.9169777421484484),
 (0.0001, 0.9169777421484484),
 (0.001, 0.91719513345279624),
 (0.01, 0.91502169197396965),
 (0.1, 0.91263085919079501),
 (1, 0.91001839102140902),
 (10, 0.89567763840422532),
 (100, 0.35036782042818071)]

**TODO** why would we be getting the same low misclassification error with so many of the low lambda values? Try with diff maxiters to see if it changes things? Try with othere param changes? Think about it generally and see what else could be causing this? Would it make sense that lambda should basically be zero? So we don't penalize anything?

**TODO** calc accuracy and misclassification error with best lambda. should I do this with all of the data? Prob not. I should prob do it with a holdout set. If the latter's true, should I be holding out that set above? Think it through and implement.